ResNet 50 output sizes of stages

In [1]:
from mmdet.models import ResNet
import torch
self = ResNet(depth=50)
self.eval()
inputs =  torch.rand(1, 3, 224, 224)
level_outputs = self.forward(inputs)
for level_out in level_outputs:
    print(tuple(level_out.shape))

c:\Users\lerch\anaconda3\envs\cv_project\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


(1, 256, 56, 56)
(1, 512, 28, 28)
(1, 1024, 14, 14)
(1, 2048, 7, 7)


### Test inference FeedbackNet

In [1]:
# adjust keys in checkpoint weights file
import torch
checkpoint = torch.load('checkpoints/faster_rcnn_r50_fpn_1x_voc0712_20220320_192712-54bef0f3.pth')
state_dict = checkpoint['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("backbone.","backbone.resnet.")] = state_dict.pop(key)
checkpoint['state_dict'] = state_dict
torch.save(checkpoint, 'checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth')

In [2]:
from mmdet.apis import init_detector, inference_detector
import mmcv

# Specify the path to model config and checkpoint file
config_file = 'configs/pascal_voc/faster_rcnn_r50fb_fpn_1x_voc0712.py'
checkpoint_file = 'checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth'

# build the model from a config file and a checkpoint file
#model = init_detector(config_file, checkpoint_file, device='cpu')
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# will only load it once
img = mmcv.imread("demo/demo.jpg")
result = inference_detector(model, img)
# visualize the results in a new window
model.show_result(img, result, out_file='output/result.jpg')

load checkpoint from local path: checkpoints/faster_rcnn_r50fb_fpn_1x_voc0712_20221208.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.fb_con.conv.weight, backbone.fb_con.conv.bias



c:\Users\lerch\Documents\Studium\Master\WiSe22\Project_CV\Code\MMdetectionRobustness\mmdet\datasets\utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
